<a href="https://colab.research.google.com/github/rudaaranha/solar_panel_number_calculation/blob/main/solar_panel_number_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install -q -U google-generativeai

In [58]:
import google.generativeai as genai
from math import ceil

In [86]:
# Configure Google Generative AI (substitua 'YOUR_API_KEY' com sua chave de API)
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Cálculo do consumo diário de energia


In [38]:
def calcular_consumo_diario(consumo_mensal: float) -> float:
    """Calcula o consumo médio diário e adiciona 30%."""
    fator_de_seguranca = 1.3
    numero_de_dias = 30
    return (consumo_mensal / numero_de_dias) * fator_de_seguranca

Cálculo da potência do sistema


In [39]:
def calcular_potencia_sistema(consumo_diario: float, irradiacao: float) -> float:
    """Calcula a potência do sistema solar em kW."""
    eficiencia = 0.8 # eficiência das placas de aprox 80%
    return consumo_diario / (eficiencia * irradiacao)

Cálculo do número de placas

In [59]:
def calcular_numero_placas(potencia_sistema: float) -> int:
    """Calcula o número de placas solares necessárias."""
    potencia_placa = 0.55  # Para placas de 550w - fazer p/ outros valores no futuro
    numero_de_placas = potencia_sistema / potencia_placa
    return int(ceil(numero_de_placas))

Resposta do Bot

In [53]:
generation_config = {
        "candidate_count": 1,
        "temperature": 0.5,
    }
model = genai.GenerativeModel(
        model_name='gemini-1.0-pro',
        generation_config=generation_config
    )

In [84]:
# Obtendo o consumo médio diário da residência e adicionando 30% como fator de segurança
consumo_mensal = float(input("Digite o seu consumo médio mensal em kWh: "))
consumo_diario = calcular_consumo_diario(consumo_mensal)

# Obtendo a irradiação média diária da cidade
print('Para realizar o cálculo energético e obter o número de placas solares necessárias para sua casa preciso saber a cidade e estado.')
cidade = input('Cidade: ')
estado = input('Estado: ')

response = model.generate_content(f'qual é a irradiação solar da cidade de {cidade} localizada no estado {estado} em kWh/m**2 * dia')

try:
  # Tenta extrair o valor assumindo o formato "irradiacao = valor"
  irradiacao_str = response.text.split(" = ")[1].split(" ")[0].replace(",", ".").strip()
  irradiacao = float(irradiacao_str)

except IndexError:
  # Se o formato não for o esperado, solicita ao modelo a resposta no formato correto
  response = model.generate_content(f'qual é a irradiação solar da cidade de {cidade} localizada no estado {estado} em kWh/m**2 * dia? Responda no formato "irradiacao = valor".')
  irradiacao_str = response.text.split(" = ")[1].split(" ")[0].replace(",", ".").strip()
  irradiacao = float(irradiacao_str)

# Calculando a potência e o número de placas necessário
potencia_sistema = calcular_potencia_sistema(consumo_diario, irradiacao)
numero_placas = calcular_numero_placas(potencia_sistema)

# Resposta final do bot
final_response = model.generate_content(f'Esse é um projeto para calcular o numero de placas solares de uma residência. faça um pequeno texto mostrando o {numero_placas} calculado. lembrando que o consumo médio é mensal e não anual')
print(final_response.text)

Digite o seu consumo médio mensal em kWh: 250
Para realizar o cálculo energético e obter o número de placas solares necessárias para sua casa preciso saber a cidade e estado.
Cidade: campina grande
Estado: campina grande
Para determinar o número de placas solares necessárias para uma residência, é crucial considerar o consumo médio mensal de energia.

Para uma residência com um consumo médio mensal de 500 kWh, por exemplo, seriam necessárias cerca de 5 placas solares. Cada placa solar produz aproximadamente 100 kWh mensais, portanto, 5 placas forneceriam 500 kWh, atendendo às necessidades energéticas da residência.
